In [ ]:
import numpy as np 
from typing import Tuple


In [ ]:
class shapeBundle: 
  """_summary_
    Eventually, A shape bundle will become the main Tensor Type, which is a shape, stride, 
    an arbitrary depth parathesization, and a mem pointer with mem packing info. This type will live 
    in some layout category augmented with Mem info type shit. 
    
    For now, we stick with abbot paper (CHAD FUCKBOY) and our type is just a shape, 
    with at most depth 2 paranthesization and we will use a simple data structure, 
    where if a shape is a list of N things indexed 0 to N-1, there is a bar already at 0 and at N-1
    if a bar is put at index i, this means that it is at position i+1 
    so if Prev-bar is at position j we take j+1 to i inside the paranthesis
    
  """
  def __init__ (self, shape:Tuple, bars:Tuple): 
    self.rank = len(shape)
    self.shape = shape 
    self.bars